In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import time
from sklearn.model_selection import train_test_split
from auxiliar_func import preprocessing

In [4]:
df = pd.read_csv('Census-Income-KDD.csv')

df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

df_train = preprocessing(df_train)
df_test = preprocessing(df_test)